In [1]:
import h5py

modelID = "record_2_model_88vVF_"
weightsFileName = "../model_files/regression_jul3_strands_augmented/" + modelID + "modelWeights.h5"
weights = h5py.File(weightsFileName, 'r')
print weights.keys()

[u'activation_1', u'activation_2', u'activation_3', u'batchnormalization_1', u'batchnormalization_2', u'batchnormalization_3', u'batchnormalization_4', u'batchnormalization_5', u'convolution1d_1', u'convolution1d_2', u'convolution1d_3', u'dense_1', u'dense_2', u'dense_3', u'dropout_1', u'dropout_2', u'flatten_1', u'maxpooling1d_1', u'maxpooling1d_2', u'maxpooling1d_3', u'prelu_1', u'prelu_2']


In [2]:
import numpy as np

layer = 'convolution1d_1'
# print np.array(weights[layer][str(layer + '_W')])
conv1_weights = np.array(weights[layer][str(layer + '_W')])
conv1_biases = np.array(weights[layer][str(layer + '_b')])
print conv1_weights.shape
print conv1_biases.shape

(19, 1, 4, 100)
(100,)


In [3]:
val_data_path = "../hdf5files/regression_jul3_strands_augmented_rep2only/valid_data.hdf5"
val_data = h5py.File(val_data_path)

In [4]:
print val_data['X']['sequence'].shape

(205984, 2000, 4)


In [32]:
import time
t0 = time.time()

numSequences = 1e5

filter_length = conv1_weights.shape[0]
stride = filter_length / 2
val_sequences = []
cnt = 0
for seq in val_data['X']['sequence']:
    if len(val_sequences) > numSequences:
        break
    seqLen = int(np.sum(seq))
    
    sliceIndices = np.arange(1000 - seqLen/2, 1000 + seqLen/2 - filter_length, stride)
    slicer = lambda t: [t, t + filter_length]
    sliceIndices = np.swapaxes(np.array(slicer(sliceIndices)), 0, 1)
    
    val_sequences += [seq[iin : iout] for iin, iout in sliceIndices]
    if len(val_sequences) / 1000 > cnt / 1000:
        cnt += len(sliceIndices)
        print "Took ~" + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt-1000) + " to " + str(cnt)
    else:
        cnt += len(sliceIndices)
#     if len(val_sequences) % 1000 == 0:
#             cnt = len(val_sequences)
#             print "Took " + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt-1000) + " to " + str(cnt)
            
#     for idx in startIndices:
#         val_sequences.append(seq[idx : idx + filter_length])
#         if len(val_sequences) % 1000 == 0:
#             cnt = len(val_sequences)
#             print "Took " + str(round(time.time() - t0, 4)) + "s for sequences " + str(cnt-1000) + " to " + str(cnt)

Took ~2.0056s for sequences 98 to 1098
Took ~3.0268s for sequences 1060 to 2060
Took ~3.7409s for sequences 2057 to 3057
Took ~4.8142s for sequences 3084 to 4084
Took ~6.3423s for sequences 4015 to 5015
Took ~7.5925s for sequences 5062 to 6062
Took ~8.7622s for sequences 6011 to 7011
Took ~9.9333s for sequences 7003 to 8003
Took ~10.9842s for sequences 8070 to 9070
Took ~12.3798s for sequences 9106 to 10106
Took ~13.5531s for sequences 10022 to 11022
Took ~14.8428s for sequences 11104 to 12104
Took ~16.2545s for sequences 12092 to 13092
Took ~17.0817s for sequences 13006 to 14006
Took ~18.1439s for sequences 14047 to 15047
Took ~19.4398s for sequences 15066 to 16066
Took ~20.6167s for sequences 16054 to 17054
Took ~21.4425s for sequences 17000 to 18000
Took ~22.7375s for sequences 18016 to 19016
Took ~23.5656s for sequences 19109 to 20109
Took ~25.2063s for sequences 20130 to 21130
Took ~26.1323s for sequences 21091 to 22091
Took ~27.1774s for sequences 22084 to 23084
Took ~28.3405s fo

In [11]:
val_sequences = np.array(val_sequences)
print val_sequences.shape

(48488, 19, 4)
